# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
df1 = pd.read_csv('homepage_actions.csv')
df1.sort_values(by='id')

,timestamp,id,group,action
4390,2016-11-26 14:38:47.682931,182988,experiment,view
5639,2016-12-14 13:19:00.940677,182994,control,view
5640,2016-12-14 13:20:29.564080,182994,control,click
376,2016-09-29 23:04:42.722325,183089,control,view
6568,2016-12-27 10:13:58.808014,183136,experiment,view
...,...,...,...,...
7389,2017-01-07 11:50:17.313666,937108,control,view
5254,2016-12-09 07:05:04.965876,937139,experiment,view
5255,2016-12-09 07:06:05.219267,937139,experiment,click
3494,2016-11-14 08:01:25.774204,937217,control,click


In [3]:
ctrl = df1[df1['group'] == 'control'].copy()
expr = df1[df1['group'] == 'experiment'].copy()
c_ids = ctrl['id'].unique()
e_ids = expr['id'].unique()

In [4]:
counter = 0
for c_id in c_ids:
    if c_id in e_ids:
        counter += 1
        print(c_id + "found in both groups!")
print(f"Number of ids found in both groups: {counter}")

Number of ids found in both groups: 0


In [5]:
for ids, data in [(c_ids, ctrl), (e_ids, expr)]:
    for i in ids:
        test_df = data[data['id']==i]
        actions = list(test_df['action'])
        if 'view' not in actions:
            print(f'Control id {i}: no view!')
        if len(test_df) > 2:
            print(f'Control id {i}: Too many items!')

No output means there are no clickers who didn't view, and there aren't any id's that had more than 2 actions (view, click)

In [6]:
print(f"Number of ids: Ctrl={len(c_ids)}, Expr={len(e_ids)}")
print(f"Number of ids who clicked: Ctrl={len(ctrl[ctrl['action']=='click'])}, Expr={len(expr[expr['action']=='click'])}")

Number of ids: Ctrl=3332, Expr=2996
Number of ids who clicked: Ctrl=932, Expr=928


In [7]:
c_clicks = {'id': [], 'clicked': []}
e_clicks = {'id': [], 'clicked': []}

for ids, data, clicks in [(c_ids, ctrl, c_clicks), (e_ids, expr, e_clicks)]:
    for i in ids:
        clicks['id'].append(i)
        if len(data[data['id']==i]) == 2:
            clicks['clicked'].append(1)
        else:
            clicks['clicked'].append(0)

df_c_clicks = pd.DataFrame.from_dict(c_clicks)
df_e_clicks = pd.DataFrame.from_dict(e_clicks)
display(df_c_clicks.head(), df_e_clicks.tail())

,id,clicked
0,671993,0
1,560027,0
2,281985,0
3,407864,0
4,342984,0


,id,clicked
2991,540466,0
2992,615849,1
2993,192060,0
2994,755912,0
2995,458115,0


In [8]:
df_c_clicks['clicked'].value_counts()

0    2400
1     932
Name: clicked, dtype: int64

In [9]:
df_e_clicks['clicked'].value_counts()

0    2068
1     928
Name: clicked, dtype: int64

In [10]:
display(df_c_clicks.describe(), df_e_clicks.describe())

,id,clicked
count,3332.000000,3332.000000
mean,562476.361345,0.279712
std,218569.808436,0.448925
min,182994.000000,0.000000
25%,378175.250000,0.000000
50%,562324.500000,0.000000
75%,753339.750000,1.000000
max,937217.000000,1.000000


,id,clicked
count,2996.000000,2996.000000
mean,565741.895527,0.309746
std,219180.840003,0.462466
min,182988.000000,0.000000
25%,369685.000000,0.000000
50%,570227.500000,0.000000
75%,762061.000000,1.000000
max,937139.000000,1.000000


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [11]:
import flatiron_stats as fs

In [12]:
fs.p_value_welch_ttest(df_c_clicks['clicked'], df_e_clicks['clicked'])

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [13]:
#Your code here
control_rate = df_c_clicks['clicked'].mean()
expected_clicks = control_rate * len(df_e_clicks)
print(f"Expected clicks in experimental group, assuming null hypothesis: {expected_clicks}")

Expected clicks in experimental group, assuming null hypothesis: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [14]:
#Your code here
actual_clicks = len(df_e_clicks[df_e_clicks['clicked'] == 1])
diff_clicks = actual_clicks - expected_clicks
n = len(df_e_clicks)
p = control_rate
variance = n * p * (1-p)
sd = np.sqrt(variance)

z_score = diff_clicks / sd
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [15]:
#Your code here

stats.norm.sf(z_score)

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [16]:
# This result, while the p-value is quite a bit smaller (0.0001 vs. 0.0044 earlier)
# seems to line up, and both would lead us to reject the null hypothesis.
# Thus, the new homepage seems to have a significant effect.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.